In [ ]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
d_code=pd.read_csv('/content/속성_D_코드.csv',index_col=0).T.to_dict()
h_code=pd.read_csv('/content/속성_H_코드.csv',index_col=0).T.to_dict()
l_code=pd.read_csv('/content/속성_L_코드.csv',index_col=0).T.to_dict()

일단 데이터중에서 날짜와 각 개인번호의 경우 명목형에 너무 많은 분류가있어서 굳이 의미없다는 생각에 제거하였고

person_prefer_f와 person_prefer_g의 경우 모두 같은 값을 가지고있어서 데이터에서 제외하였습니다.(-_-님의 코드 참고)

In [ ]:
#데이터중 일부 변수 제거

train = train.drop(['id', 'contents_open_dt','person_rn', 'contents_rn',
# 'person_prefer_d_1','person_prefer_d_2','person_prefer_d_3',
# 'contents_attribute_d','person_prefer_h_1','person_prefer_h_2','person_prefer_h_3','contents_attribute_h','contents_attribute_l',
'person_prefer_f','person_prefer_g'], axis=1) 


test = test.drop(['id', 'contents_open_dt','person_rn', 'contents_rn',
# 'person_prefer_d_1','person_prefer_d_2','person_prefer_d_3',
# 'contents_attribute_d','person_prefer_h_1','person_prefer_h_2','person_prefer_h_3','contents_attribute_h','contents_attribute_l',
'person_prefer_f','person_prefer_g'], axis=1)

# print(train.head())
train['d_l_match_yn']=train['d_l_match_yn'].replace([True,False],[1,0])
train['d_m_match_yn']=train['d_m_match_yn'].replace([True,False],[1,0])
train['d_s_match_yn']=train['d_s_match_yn'].replace([True,False],[1,0])
train['h_l_match_yn']=train['h_l_match_yn'].replace([True,False],[1,0])
train['h_m_match_yn']=train['h_m_match_yn'].replace([True,False],[1,0])
train['h_s_match_yn']=train['h_s_match_yn'].replace([True,False],[1,0])

test['d_l_match_yn']=test['d_l_match_yn'].replace([True,False],[1,0])
test['d_m_match_yn']=test['d_m_match_yn'].replace([True,False],[1,0])
test['d_s_match_yn']=test['d_s_match_yn'].replace([True,False],[1,0])
test['h_l_match_yn']=test['h_l_match_yn'].replace([True,False],[1,0])
test['h_m_match_yn']=test['h_m_match_yn'].replace([True,False],[1,0])
test['h_s_match_yn']=test['h_s_match_yn'].replace([True,False],[1,0])

이번 colab에서 gpu를 가져다 썼는데 정말 더 빠른지는 아직 체감하지 못했습니다...

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(42)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(42)


데이터 상태 확인과 기존에 너무 숫자가 큰탓에 모델이 안돌아가는것인가? 했지만

아니라는 사실을 깨닫고 다시 원상태로 해놨습니다. 그런데 변수의 숫자 크기에따라 모델의 성능이 조금 차이난다는 사실을 경험적으로 알수있었습니다. 전부 0과 10사이로 맞추는게 좀더 모델성능이 높게 나왔습니다. 물론 100과 10이 같은 변수가 된다는것을 감안 해서도요.

이번이 처음인지라 이번케이스에서만 그런건지 아님 다른케이스에서도 그런건지는 잘 모르겠습니다.

In [ ]:
#DataLoader의사용

x = train.iloc[:, :-1]
y = train.iloc[:, -1]

x= x.to_numpy()
print(x[0])
print(y[0])
# x=np.where(x > 1000, x/1000, x)
# x=np.where(x > 100, x/100, x)
# x=np.where(x > 10, x/10, x)
# x=x/10
x=torch.Tensor(x)

print(x.size())
print(x[0][1])
y=torch.Tensor(y)

print(x)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)


dataset_train = TensorDataset(x_train, y_train)
dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)


dataset_valid = TensorDataset(x_valid, y_valid)
dataloader_valid = DataLoader(dataset_valid, batch_size=64, shuffle=True)


x_test=test.iloc[:]
x_test=x_test.to_numpy()
# x_test=np.where(x_test > 1000, x_test/1000, x_test)
# x_test=np.where(x_test > 100, x_test/100, x_test)
# x_test=np.where(x_test > 10, x_test/10, x_test)
# x_test=x_test/10
print(x_test.shape)
x_test=torch.FloatTensor(x_test)

LogisticeRegression모델입니다.
데이터가 워낙 많은지라 깊을수록 좋다고 판단해서 9층까지 넣었는데 5~9층 모두 비슷한 성능을 보여주었습니다.

깊게넣느라 LeaKReLU를 사용하고 마지막에 sigmoid를 사용했습니다.

In [ ]:



class LogisticRegression(nn.Module):
    
    def __init__(self):
        super(LogisticRegression, self).__init__()

        # self.linear = nn.Sequential(
        self.linear1=nn.Linear(28,16)
        self.batch_normal1=nn.BatchNorm1d(16)
        self.LeakRelu=nn.LeakyReLU()

        self.linear2=nn.Linear(16, 16)
        self.batch_normal2=nn.BatchNorm1d(16)
        self.LeakRelu=nn.LeakyReLU()

        self.linear3=nn.Linear(16, 16)
        self.batch_normal3=nn.BatchNorm1d(16)
        self.LeakRelu=nn.LeakyReLU()

        self.linear4=nn.Linear(16, 16)
        self.batch_normal4=nn.BatchNorm1d(16)
        self.LeakRelu=nn.LeakyReLU()

        self.linear5=nn.Linear(16, 16)
        self.batch_normal5=nn.BatchNorm1d(16)
        self.LeakRelu=nn.LeakyReLU()

        self.linear6=nn.Linear(16, 16)
        self.batch_normal6=nn.BatchNorm1d(16)
        self.LeakRelu=nn.LeakyReLU()

        self.linear7=nn.Linear(16, 1)
        self.batch_normal7=nn.BatchNorm1d(1)
        self.LeakRelu=nn.LeakyReLU()

        self.sigmoid=nn.Sigmoid()    
        torch.nn.init.xavier_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
        torch.nn.init.xavier_uniform_(self.linear3.weight)
        torch.nn.init.xavier_uniform_(self.linear4.weight)
        torch.nn.init.xavier_uniform_(self.linear5.weight)
        torch.nn.init.xavier_uniform_(self.linear6.weight)
        torch.nn.init.xavier_uniform_(self.linear7.weight)
        # )        
#  self.linear2,self.batch_normal2, self.LeakRelu, 
        self.model=nn.Sequential(self.linear1, self.batch_normal1, self.LeakRelu,
                                 self.linear2,self.batch_normal2, self.LeakRelu, 
                                 self.linear3,self.batch_normal3,self.LeakRelu,
                                 self.linear4, self.batch_normal4,self.LeakRelu,
                                 self.linear5, self.batch_normal5,self.LeakRelu,
                                 self.linear6, self.batch_normal6,self.LeakRelu,
                                 self.linear7, self.batch_normal7,self.LeakRelu,
                                 self.sigmoid)

    def forward(self, x):
        return self.model(x)

model1=LogisticRegression().to(device)

optimzer는 Adam을 사용했습니다. 일단 가장 많이들 사용한다고 하더군요

In [ ]:
criterion1=nn.CrossEntropyLoss().to(device)
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
print(model1.parameters)


모델 을 돌리는 코드입니다.

밑에서 시각화를 해보려했고 가장 높은 epoch을 선택하기위해 torch.save()함수를 사용했습니다.

In [ ]:
nb_epochs = 50
accuracy_list_train=[]
accuracy_list_valid=[]
f1_list_train=[]
f1_list_valid=[]
accuracy_mean=0
f1_mean=0

# x_valid, y_valid=dataset_valid

x_valid=x_valid.to(device)
y_valid=y_valid.to(device)

for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader_train):

        x_train, y_train = samples

        x_train=x_train.to(device)
        y_train=y_train.to(device)


        # H(x) 계산
        hypothesis = model1(x_train)
        
        
    
        # cost 계산
        # print(y_train.size())
        hypothesis=hypothesis.squeeze()
        # print(hypothesis.size())
        # print(hypothesis)
        # print(y_train)
        cost = F.binary_cross_entropy(hypothesis, y_train)

        # cost로 H(x) 개선
        optimizer1.zero_grad()
        cost.backward()
        optimizer1.step()

        bigo=torch.FloatTensor([0.5]).to(device)

        prediction = hypothesis >= bigo # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        accuracy_mean= accuracy+accuracy_mean
        f1 = f1_score(y_train.cpu(), prediction.float().cpu())
        f1_mean=f1 + f1_mean

        # 20번마다 로그 출력
        if epoch % 10 == 0:
          if batch_idx %1000 == 0:
              prediction = hypothesis >= bigo # 예측값이 0.5를 넘으면 True로 간주
              correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
              accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
              f1 = f1_score(y_train.cpu(), prediction.float().cpu())
              print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f} Accuracy {:2.2f}%  f1 score : {:2.6f}'.format( # 각 에포크마다 정확도를 출력
                  epoch, nb_epochs, batch_idx+1, len(dataloader_train), cost.item(), accuracy * 100, f1
              ))

    accuracy_mean=accuracy_mean/len(dataloader_train)
    accuracy_list_train.append(accuracy_mean)
    f1_mean=f1_mean/len(dataloader_train)
    f1_list_train.append(f1_mean)

    valid_hypothesis = model1(x_valid)
    valid_hypothesis=valid_hypothesis.squeeze()
    valid_prediction = valid_hypothesis >= bigo # 예측값이 0.5를 넘으면 True로 간주
    valid_correct_prediction = valid_prediction.float() == y_valid # 실제값과 일치하는 경우만 True로 간주
    valid_accuracy = valid_correct_prediction.sum().item() / len(valid_correct_prediction) # 정확도를 계산
    # print(valid_hypothesis)
    # print(valid_prediction)
    
    # print(valid_accuracy)
    # print(len(valid_correct_prediction))
    # print(valid_correct_prediction.sum().item())
    valid_f1 = f1_score(y_valid.cpu(), valid_prediction.float().cpu())

    accuracy_list_valid.append(valid_accuracy)
    f1_list_valid.append(valid_f1)
    torch.save(model1.state_dict(), f'{epoch:04d}.pt')


모델의 epoch당 정확도와 f1_score을 시각화 한것입니다. 각각을 담은 리스트는 바로 위의 코드에 같이 적혀있습니다.

In [ ]:
import matplotlib.pyplot as plt


t=range(0,nb_epochs+1)
print(len(accuracy_list_train))
print(len(accuracy_list_valid))

plt.plot(t,accuracy_list_train,'r--',t,accuracy_list_valid,'g-')


plt.show()

print(accuracy_list_train)
print(accuracy_list_valid)

In [ ]:
plt.plot(t,f1_list_train,'r--',t,f1_list_valid,'g-')

plt.show()

print(f1_list_train)
print(f1_list_valid)

결과 제출 코드입니다. 여기서 가장 성능이 좋았다고 판단되는 epoch의 모델을 가져와서 결과를 냅니다.

train(빨간줄) 보다 valid(녹색)줄의 성능이 더 좋은것을 가져와야 할것입니다.

In [ ]:
result=pd.read_csv('/content/sample_submission.csv')

model2=LogisticRegression()
model2.load_state_dict(torch.load('/content/0010.pt'))

x_test=x_test.to(device)
hypothesis=model2(x_test)
print(hypothesis)
prediction = hypothesis >= torch.FloatTensor([0.5]).to(device) # 예측값이 0.5를 넘으면 True로 간주

print(prediction)

prediction = torch.squeeze(prediction)
prediction=prediction.cpu()
prediction=np.array(prediction)
prediction=pd.Series(prediction)

prediction=prediction.replace([True,False],[1,0])

result['target']=prediction
result.to_csv('result.csv',index=False)

딥러닝 모델을 직접 만들어보고 싶었는데 이번기회에 간단하게나마 하나 만들어 볼수있어서 굉장히 좋은 기회였습니다.

어떤식으로 해야 위의 성능이 좀더 좋아질수 있을까 여러 측면으로 고민해보고 다른 모델은 어떠할지 비교해봐야 겠습니다.